# 1. Imports packages

In [165]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from library.sb_utils import save_file

ModuleNotFoundError: No module named 'library'

# 2. Load Data

In [92]:
train = pd.read_csv("../store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("../store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv("../store-sales-time-series-forecasting/stores.csv")
oil = pd.read_csv("../store-sales-time-series-forecasting/oil.csv")
holidays_events = pd.read_csv("../store-sales-time-series-forecasting/holidays_events.csv")
transactions = pd.read_csv("../store-sales-time-series-forecasting/transactions.csv").sort_values(["store_nbr", "date"])


# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)


Take a quick look of the data

In [93]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [94]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [15]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [8]:
transactions.head()

,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520


In [11]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


# 3. Data Cleaning - transactions

Let's start with the transaction data

In [95]:
transactions.head()

,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520


In [96]:
transactions.describe()

,store_nbr,transactions
count,83488.000000,83488.000000
mean,26.939237,1694.602158
std,15.608204,963.286644
min,1.000000,5.000000
25%,13.000000,1046.000000
50%,27.000000,1393.000000
75%,40.000000,2079.000000
max,54.000000,8359.000000


In [117]:
transactions.dtypes

date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object

In [98]:
# Check for any missing values
transactions.isna().any()

date            False
store_nbr       False
transactions    False
dtype: bool

In [77]:
# Checking for duplicates
duplicated = transactions.duplicated()

In [78]:
transactions[duplicated]

,date,store_nbr,transactions


# 3. Data Cleaning - stores

In [61]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [62]:
stores.dtypes

store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object

In [63]:
stores.isna().any()

store_nbr    False
city         False
state        False
type         False
cluster      False
dtype: bool

In [80]:
duplicated = stores.duplicated()

In [81]:
stores[duplicated]

,store_nbr,city,state,type,cluster


# 3. Data Cleaning - train

In [64]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [116]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion           float16
dtype: object

In [115]:
train.onpromotion = train.onpromotion.astype("float16")

In [68]:
train.isna().any()

id             False
date           False
store_nbr      False
family         False
sales          False
onpromotion    False
dtype: bool

In [82]:
duplicated = train.duplicated()

In [83]:
train[duplicated]

,id,date,store_nbr,family,sales,onpromotion


# 3. Data Cleaning - test

In [69]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [74]:
test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
onpromotion           float16
dtype: object

In [72]:
test.describe()

,id,store_nbr,onpromotion
count,2.851200e+04,28512.000000,28512.000000
mean,3.015144e+06,27.500000,6.965383
std,8.230850e+03,15.586057,20.683952
min,3.000888e+06,1.000000,0.000000
25%,3.008016e+06,14.000000,0.000000
50%,3.015144e+06,27.500000,0.000000
75%,3.022271e+06,41.000000,6.000000
max,3.029399e+06,54.000000,646.000000


In [73]:
test.onpromotion = test.onpromotion.astype("float16")

In [75]:
test.isna().any()

id             False
date           False
store_nbr      False
family         False
onpromotion    False
dtype: bool

In [84]:
duplicated = test.duplicated()

In [85]:
test[duplicated]

,id,date,store_nbr,family,onpromotion


# 3. Data Cleaning - oil

In [86]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [87]:
oil.dtypes

date           object
dcoilwtico    float64
dtype: object

In [88]:
oil.isna().any()

date          False
dcoilwtico     True
dtype: bool

In [89]:
duplicated = oil.duplicated()

In [99]:
oil[duplicated]

,date,dcoilwtico


# 4. Data Merging

In [102]:
transactions.head()

,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520


In [103]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [104]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [139]:
temp = train.merge(transactions, on=['date','store_nbr'], how='left')

In [140]:
temp.head()

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0.0,NaN


In [141]:
df = temp.merge(stores, on='store_nbr', how='left')

In [148]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,0.0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0.0,0.0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0.0,0.0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,0.0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0.0,0.0,Quito,Pichincha,D,13


In [146]:
df['transactions'] = df2['transactions'].fillna(0)

In [147]:
df.isnull().values.any()

False

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   float16       
 6   transactions  float64       
 7   city          object        
 8   state         object        
 9   type          object        
 10  cluster       int64         
dtypes: datetime64[ns](1), float16(1), float64(2), int64(3), object(4)
memory usage: 257.6+ MB


In [163]:
df.shape

(3000888, 11)

In [164]:
# save the data to a new csv file
datapath = '../data'
save_file(df, 'store_sales_transactions.csv', datapath)

NameError: name 'save_file' is not defined

# 5. Summary

### Key Processes:

- Load 7 files: train, test, transactions, stores, oil, holidays_events.
- None of the dateset has any missing values or duplicates.
- Converted "date" field in train and transactions to datetime type.
- Created Df: Merged transactions, train, and stores to one dataset
- Df has a shape of (3000888, 11). 
